In [1]:
# k crossover d，close > ema 8, ema 8 > ema 18, ema 18 > 38, take profit atr, stop loss atr
# Variables :
# time - 15m, 1h
# start time - 1609459200000, 1641028611000, 1672564611000, 1654045261000, 1651366861000 (2021, 2022, 2023)
# start_time = 1635743450000 2021/11 end_time = 1654060250000 2022/6 BEAR

In [2]:
# ! conda install -c conda-forge ta --yes

In [3]:
import requests
import pandas as pd
import ta
import datetime as dt
import numpy as np
import time
import itertools
import os

In [4]:
from binance.client import Client
from dotenv import load_dotenv

# Get the path to the current directory
current_directory = os.getcwd()

# Specify the path to the .env file relative to the current directory
dotenv_path = os.path.join(current_directory, '.env')

# Load the environment variables from the .env file
load_dotenv(dotenv_path)

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

In [5]:
# start_time_arr = [1672531200000, 1651366861000]

interval_arr = ['1h', '15m']

# how kd is crossed
kd_dir_arr = ['Any', 'Up']

sl_atr_arr = [1, 3, 5]

tp_atr_arr = [3, 5, 9, 12]

# stop loss determine candlestick
sl_det_arr = ['Close']

In [6]:
timezone = 8
# symbol = 'nearusdt'
symbol_arr = ['nearusdt', 'ftmusdt', 'maticusdt', 'sandusdt', 'woousdt', 'apeusdt', 'dydxusdt', 'atomusdt']
start_time_d = 1609459200000
end_time = round(time.time() * 1000)
# end_time = 1654060250000 
# step between timestamps in milliseconds
step = 1000

In [7]:
dataframes = {}
def create_raw(symbol_arr, interval_arr, start_time, end_time, step):
        
    for interval in interval_arr:
        for symbol in symbol_arr:
            # Fetch the data using batch requests
            data = []    
            while start_time < end_time:
                limit = min(step, end_time - start_time + 1)  # Adjust the limit for the last batch
                response = client.get_klines(symbol=symbol.upper(), interval=interval, limit=limit, startTime=start_time)

                if len(response) == 0:
                    break  # No more data available, exit the loop
                data.extend(response)
                start_time = response[-1][0] + 1

            # Convert the data to a DataFrame
            columns = [
                "Open_Time", "Open", "High", "Low", "Close", "Volume", "Close_Time",
                "Quote asset volume", "Number of trades", "Taker buy base asset volume",
                "Taker buy quote asset volume", "Ignore"
            ]
            raw_df = pd.DataFrame(data, columns=columns)   

            raw_df = raw_df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']]

            dataframes[f'df_{interval}_{symbol}'] = raw_df
            start_time = start_time_d

In [8]:
loop_start_time = time.time()
create_raw(symbol_arr, interval_arr, start_time_d, end_time, step)
loop_end_time = time.time()
print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")
print(dataframes)

Time taken to execute for loop: 66.85672736167908 seconds
{'df_1h_nearusdt':            Open_Time        Open       Close        High         Low  \
0      1609459200000  1.39070000  1.45170000  1.48000000  1.38950000   
1      1609462800000  1.44990000  1.47650000  1.48870000  1.44300000   
2      1609466400000  1.47920000  1.48730000  1.48970000  1.45520000   
3      1609470000000  1.48730000  1.44500000  1.48860000  1.44310000   
4      1609473600000  1.44700000  1.44680000  1.45590000  1.44000000   
...              ...         ...         ...         ...         ...   
21164  1685700000000  1.59300000  1.59700000  1.60100000  1.59200000   
21165  1685703600000  1.59700000  1.60100000  1.60700000  1.59600000   
21166  1685707200000  1.60100000  1.59500000  1.60500000  1.58800000   
21167  1685710800000  1.59600000  1.58900000  1.61100000  1.58900000   
21168  1685714400000  1.58900000  1.59000000  1.59500000  1.58800000   

                 Volume  
0      1037332.44000000  
1     

In [9]:
def get_klines(symbol, interval, start_time, end_time):
    global dataframes
    df = dataframes[f'df_{interval}_{symbol}']
    df = df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']].astype(float)
    df = df.set_index('Open_Time')

    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)
    df = df[~df.index.duplicated(keep='first')]

    return df

In [10]:
def check_cross(df, kd_dir):
    up = df['slow_k'] > df['slow_d']
    down = df['slow_k'] < df['slow_d']
    if kd_dir == 'Up':
        return up.diff() & up
    if kd_dir == 'Any':
        return up.diff()
    if kd_dir == 'Down':
        return down.diff() & down


def indicators(df, kd_dir):

# ema
    for i in (8, 18, 38):
        df['ema_'+str(i)] = ta.trend.ema_indicator(df.Close, window=i)

# atr
    df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)
    
# rsi
    rsi_int = 14
    df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window = rsi_int).rsi()

# kd

    df['slow_k']= ta.momentum.stoch(df['High'], df['Low'], df['Close'], 17, 5)
    df['slow_d'] = ta.momentum.stoch_signal(df['High'], df['Low'], df['Close'], 17, 5)
    
# kd cross
    df['kd_cross'] = check_cross(df, kd_dir)

In [11]:
def conditions(df):

    # c1
    df['c1'] = df['kd_cross']
    # c2
    df['c2'] = df['Close'] >= df['ema_8']
    # c3
    df['c3'] = df['ema_8'] >= df['ema_18']
    # c4
    df['c4'] = df['ema_18'] >= df['ema_38']

    # signal
    df['signal'] = df['c1'] & df['c2'] & df['c3'] & df['c4']
    
    # open_entry
    df['open_entry'] = df['signal'].shift()

In [12]:
def entries(df, sl_atr, tp_atr, sl_det):

    in_position = False
    stop_loss = np.nan
    take_profit = np.nan
    close_val = df['Close']
    atr_val = df['atr']
    
    df['position'] = ''  # Create an empty column for position

    for index, row in df.iterrows():

        if index == 0:
            continue

        elif row['open_entry']:

            df.at[index, 'entry_p'] = close_val.shift(1).at[index]
            df.at[index, 'stop_loss'] = close_val.shift(1).at[index] - sl_atr * atr_val.shift(1).at[index]
            df.at[index, 'take_profit'] = close_val.shift(1).at[index] + tp_atr * atr_val.shift(1).at[index]
            df.at[index, 'position'] = 'Buy'
            in_position = True
            stop_loss = df.at[index, 'stop_loss']
            take_profit = df.at[index, 'take_profit']


        # 吃筍
        #-----------------------------重要-----------------------------
        # 若用 if 寫，則有可能入場馬上吃筍，若用 elif 則一個 iteration 只會執行一次
        elif in_position and row['Close'] <= stop_loss:
            df.at[index, 'position'] = 'Stop'
            in_position = False
            stop_loss = np.nan
            take_profit = np.nan

        # set take profit
        elif in_position and row['High'] >= take_profit:
            df.at[index, 'position'] = 'Sell'
            in_position = False
            stop_loss = np.nan
            take_profit = np.nan
    

    # 過濾有訊號或事件發生的Ｋ線
    df = df[(df['open_entry']) | (df['position'] != '')]

In [13]:
# 部位回測
def backtest(df):

    df = df.reset_index(drop = True)
    df = df[(df['position'] == 'Buy') |
                  (df['position'] == 'Sell') |
                  (df['position'] == 'Stop')]

    # 一次進場多少單位
    pos_size = 1

    col = ['Open_Time', 'Open', 'Close', 'High', 'Low', 'atr', 'position','entry_p', 'stop_loss', 'take_profit']
    pos = df[col]
    pos = pos.reset_index(drop = True)


    for index, row in pos.iterrows():

        current_pos = 0

        # 進場
        if pos.at[index, 'position'] == 'Buy':
            pos.at[index, 'size'] = pos_size
            pos.exit_p = np.nan

        # 出場
        if pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':

            # 停利：達成條件時收盤價
            if pos.at[index, 'position'] == 'Sell':
                for i in range(index -1, -1, -1):
                    if pos.at[i, 'position'] == 'Buy':
                        pos.at[index, 'exit_p'] = pos.at[i, 'take_profit']
                    break

            # 停損：打到進場停損點（往回跌代，直到最近的'Buy'及其'stop_loss'）
            if pos.at[index, 'position'] == 'Stop':
                pos.at[index, 'exit_p'] = pos.at[index, 'Open']
#                 for i in range(index -1, -1, -1):
#                     if pos.at[i, 'position'] == 'Buy':
#                         pos.at[index, 'exit_p'] = pos.at[i, 'stop_loss']
#                     break

            # 計算每次出場部位大小（每次出場皆清倉）
            for i in range(index -1, -1, -1):
                if pos.at[i, 'position'] == 'Buy':
                    current_pos += pos.at[i, 'size']
                    if i == 0:
                        pos.at[index, 'size'] = -current_pos
                    else:
                        continue
                else:
                    pos.at[index, 'size'] = -current_pos
                    current_pos = 0
                    break


    # 計算部位價值
    for index, row in pos.iterrows():
        if pos.at[index, 'position'] == 'Buy':
            pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'entry_p'], 4)
        elif pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':
            pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'exit_p'], 4)


    # 若最後一筆為 Buy，移除該單，迭代驗證
    for index, row in pos.iloc[::-1].iterrows():
        if row['position'] == 'Buy':
            pos = pos.drop(index)
        else:
            break


    # 手續費、滑點、價差
    fee = 0.05 / 100
    amt_abs_sum = pos.amt.abs().sum()
    ttl_fee = amt_abs_sum * fee

    # 損益
    leverage = 10
    ttl_profit = -pos.amt.sum() - ttl_fee

    
    # 計算獲利/虧損次數
    agg_amts = []

    for i in range(len(pos) - 1, -1, -1):

        if pos.loc[i, 'position'] in ['Stop', 'Sell']:

            # look out for the + sign
            total_amt = pos.loc[i, 'amt'] + np.absolute(pos.loc[i, 'amt']) * fee

            # iterate backwards from the current row until reaching another 'Stop' or 'Sell'
            # watch out for the + in total_amt += trading_fee
            j = i - 1
            while j >= 0 and pos.loc[j, 'position'] not in ['Stop', 'Sell']:
                total_amt += pos.loc[j, 'amt']
                trading_fee = np.absolute(pos.loc[j, 'amt']) * fee
                total_amt += trading_fee
                j -= 1

            # add the aggregated amount to the list
            agg_amts.append(total_amt)

    agg_amts.reverse()

    
    # 計算進場最大部位，最大損益
    consec_entry = 0
    position_amt_sum = 0
    max_consec_entry = 0
    max_position = 0
    max_profit = 0
    max_loss = 0

    for index, row in pos.iterrows():

        if row['position'] == 'Buy':

            consec_entry += 1
            position_amt_sum += row['amt']

        elif row['position'] in ['Sell', 'Stop']:

            if consec_entry > max_consec_entry:
                max_consec_entry = consec_entry
                max_position = position_amt_sum

            position_amt_sum += row['amt']

            if -position_amt_sum > max_profit:
                max_profit = -position_amt_sum

            if -position_amt_sum < max_loss:
                max_loss = -position_amt_sum

            consec_entry = 0
            position_amt_sum = 0

        else:
            pass


    # 最大部位
    profit_per = "{:.2f}%".format(ttl_profit / (max_position/leverage) * 100)


    # 勝率
    wins = 0
    loses = 0

    for trade in agg_amts:
        if trade < 0:
            wins += 1
        elif trade > 0:
            loses += 1

    win_rate = "{:.2f}%".format(wins / (wins + loses) * 100)
    
    cumulative_values = []
    cumulative_sum = 0

    for value in agg_amts:
        cumulative_sum -= value
        cumulative_values.append(cumulative_sum)

    max_drawdown = min(cumulative_values)

    # 結果
    result = {'Profit': [round(ttl_profit, 2)],
              'Fee': [round(ttl_fee, 2)],
              'Max_Profit': [round(max_profit, 2)],
              'Max_Loss': [round(max_loss, 2)],
              'Max_Entry': [max_consec_entry],
              'Max_Position': [round(max_position, 2)],
              'Profit_%': [profit_per],
              'Win_Rate': [win_rate],
              'PF_Ratio': [round(ttl_profit/ttl_fee, 2)],
              'Max_Drawdown': [round(max_drawdown/ttl_profit, 2)]}


    result_df = pd.DataFrame(result)

    return result_df, ttl_profit, win_rate, ttl_fee

In [14]:
# iterate using start_time, under same time interval, test different variables
def run_backtest():

    results_df = pd.DataFrame()
    
    symbol_list = []
    interval_list = []
    kd_dir_list = []
    sl_atr_list = []
    tp_atr_list = []
    sl_det_list = []

    i = 0

    loop_start_time = time.time()
    
    parameters = [(symbol, interval, kd_dir, sl_atr, tp_atr, sl_det) 
                  for symbol in symbol_arr
                  for interval in interval_arr
                  for kd_dir in kd_dir_arr
                  for sl_atr in sl_atr_arr
                  for tp_atr in tp_atr_arr
                  for sl_det in sl_det_arr]
    
    for symbol, interval, kd_dir, sl_atr, tp_atr, sl_det in parameters:

        df = get_klines(symbol, interval, start_time_d, end_time)
        df = df.reset_index()

        indicators(df, kd_dir)
        conditions(df)
        entries(df, sl_atr, tp_atr, sl_det)
        
        backtest_df, ttl_profit, win_rate, fee = backtest(df)

        results_df = pd.concat([results_df, backtest_df], ignore_index = True)

        symbol_list.append(symbol)
        interval_list.append(interval)
        kd_dir_list.append(kd_dir)
        sl_atr_list.append(sl_atr)
        tp_atr_list.append(tp_atr)
        sl_det_list.append(sl_det)

        i += 1

        print(f" {i} - {symbol}, {interval}, sl: {sl_atr}, tp: {tp_atr}, {sl_det}, {round(ttl_profit, 2)}, {round(fee, 2)}, {round(ttl_profit/fee, 2)}, {win_rate} ")

    loop_end_time = time.time()

    print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")

    results_df['symbol'] = symbol_list
    results_df['interval'] = interval_list
    results_df['kd_dir'] = kd_dir_list
    results_df['sl_atr'] = sl_atr_list
    results_df['tp_atr'] = tp_atr_list
    results_df['sl_det'] = sl_det_list

    results_df = results_df[['symbol', 'interval', 'kd_dir',
                             'sl_atr', 'tp_atr', 'sl_det',
                             'Profit', 'Fee', 'Max_Profit', 'Max_Loss',
                             'Max_Entry', 'Max_Position', 'Profit_%', 'Win_Rate', 'PF_Ratio', 'Max_Drawdown'
                            ]]
    
    return results_df

In [15]:
results_df = run_backtest()
results_df = results_df.sort_values('Profit', ascending = False)
# print(results_df)

 1 - nearusdt, 1h, sl: 1, tp: 3, Close, 151.65, 11.19, 13.55, 36.60% 
 2 - nearusdt, 1h, sl: 1, tp: 5, Close, 217.48, 11.22, 19.38, 33.91% 
 3 - nearusdt, 1h, sl: 1, tp: 9, Close, 224.21, 11.23, 19.97, 33.80% 
 4 - nearusdt, 1h, sl: 1, tp: 12, Close, 224.98, 11.23, 20.04, 33.67% 
 5 - nearusdt, 1h, sl: 3, tp: 3, Close, 101.94, 11.17, 9.13, 40.69% 
 6 - nearusdt, 1h, sl: 3, tp: 5, Close, 227.05, 11.23, 20.22, 25.93% 
 7 - nearusdt, 1h, sl: 3, tp: 9, Close, 176.23, 11.2, 15.73, 22.34% 
 8 - nearusdt, 1h, sl: 3, tp: 12, Close, 182.73, 11.21, 16.31, 22.46% 
 9 - nearusdt, 1h, sl: 5, tp: 3, Close, 186.33, 11.21, 16.62, 55.45% 
 10 - nearusdt, 1h, sl: 5, tp: 5, Close, 330.6, 11.28, 29.31, 31.82% 
 11 - nearusdt, 1h, sl: 5, tp: 9, Close, 484.5, 11.36, 42.66, 20.79% 
 12 - nearusdt, 1h, sl: 5, tp: 12, Close, 471.72, 11.35, 41.55, 21.00% 
 13 - nearusdt, 1h, sl: 1, tp: 3, Close, 44.28, 6.12, 7.24, 36.89% 
 14 - nearusdt, 1h, sl: 1, tp: 5, Close, 57.17, 6.12, 9.34, 34.78% 
 15 - nearusdt, 1h, sl

 119 - maticusdt, 1h, sl: 5, tp: 9, Close, 32.71, 1.23, 26.51, 24.55% 
 120 - maticusdt, 1h, sl: 5, tp: 12, Close, 31.54, 1.23, 25.57, 23.81% 
 121 - maticusdt, 15m, sl: 1, tp: 3, Close, 26.1, 8.62, 3.03, 30.89% 
 122 - maticusdt, 15m, sl: 1, tp: 5, Close, 31.78, 8.63, 3.68, 29.33% 
 123 - maticusdt, 15m, sl: 1, tp: 9, Close, 32.02, 8.63, 3.71, 29.23% 
 124 - maticusdt, 15m, sl: 1, tp: 12, Close, 32.17, 8.63, 3.73, 29.23% 
 125 - maticusdt, 15m, sl: 3, tp: 3, Close, 47.3, 8.63, 5.48, 38.77% 
 126 - maticusdt, 15m, sl: 3, tp: 5, Close, 68.12, 8.64, 7.88, 27.73% 
 127 - maticusdt, 15m, sl: 3, tp: 9, Close, 79.42, 8.65, 9.18, 24.15% 
 128 - maticusdt, 15m, sl: 3, tp: 12, Close, 84.74, 8.65, 9.79, 23.69% 
 129 - maticusdt, 15m, sl: 5, tp: 3, Close, 38.25, 8.62, 4.44, 50.22% 
 130 - maticusdt, 15m, sl: 5, tp: 5, Close, 68.41, 8.63, 7.93, 30.86% 
 131 - maticusdt, 15m, sl: 5, tp: 9, Close, 78.33, 8.63, 9.07, 23.19% 
 132 - maticusdt, 15m, sl: 5, tp: 12, Close, 85.1, 8.64, 9.85, 22.20% 
 133 

 237 - woousdt, 15m, sl: 5, tp: 3, Close, 1.81, 0.58, 3.1, 55.62% 
 238 - woousdt, 15m, sl: 5, tp: 5, Close, 1.38, 0.58, 2.38, 32.55% 
 239 - woousdt, 15m, sl: 5, tp: 9, Close, 3.19, 0.58, 5.49, 20.97% 
 240 - woousdt, 15m, sl: 5, tp: 12, Close, 1.09, 0.58, 1.88, 20.31% 
 241 - apeusdt, 1h, sl: 1, tp: 3, Close, 52.95, 5.66, 9.35, 30.69% 
 242 - apeusdt, 1h, sl: 1, tp: 5, Close, 93.97, 5.69, 16.53, 28.95% 
 243 - apeusdt, 1h, sl: 1, tp: 9, Close, 68.4, 5.67, 12.06, 28.97% 
 244 - apeusdt, 1h, sl: 1, tp: 12, Close, 58.81, 5.67, 10.38, 29.32% 
 245 - apeusdt, 1h, sl: 3, tp: 3, Close, 130.38, 5.7, 22.86, 36.36% 
 246 - apeusdt, 1h, sl: 3, tp: 5, Close, 266.73, 5.77, 46.21, 22.81% 
 247 - apeusdt, 1h, sl: 3, tp: 9, Close, 126.3, 5.7, 22.15, 19.39% 
 248 - apeusdt, 1h, sl: 3, tp: 12, Close, 107.05, 5.69, 18.81, 18.95% 
 249 - apeusdt, 1h, sl: 5, tp: 3, Close, 104.5, 5.69, 18.36, 52.43% 
 250 - apeusdt, 1h, sl: 5, tp: 5, Close, 312.08, 5.79, 53.86, 33.33% 
 251 - apeusdt, 1h, sl: 5, tp: 9, Cl

 354 - atomusdt, 1h, sl: 3, tp: 5, Close, 161.19, 20.56, 7.84, 26.05% 
 355 - atomusdt, 1h, sl: 3, tp: 9, Close, 94.1, 20.53, 4.58, 21.16% 
 356 - atomusdt, 1h, sl: 3, tp: 12, Close, 95.75, 20.53, 4.66, 20.97% 
 357 - atomusdt, 1h, sl: 5, tp: 3, Close, 85.44, 20.52, 4.16, 51.14% 
 358 - atomusdt, 1h, sl: 5, tp: 5, Close, 357.06, 20.66, 17.28, 32.17% 
 359 - atomusdt, 1h, sl: 5, tp: 9, Close, 393.65, 20.68, 19.04, 23.68% 
 360 - atomusdt, 1h, sl: 5, tp: 12, Close, 253.25, 20.61, 12.29, 21.82% 
 361 - atomusdt, 15m, sl: 1, tp: 3, Close, 563.18, 147.12, 3.83, 32.03% 
 362 - atomusdt, 15m, sl: 1, tp: 5, Close, 664.03, 147.17, 4.51, 29.99% 
 363 - atomusdt, 15m, sl: 1, tp: 9, Close, 691.71, 147.18, 4.7, 29.64% 
 364 - atomusdt, 15m, sl: 1, tp: 12, Close, 698.13, 147.18, 4.74, 29.69% 
 365 - atomusdt, 15m, sl: 3, tp: 3, Close, 805.84, 147.18, 5.48, 36.80% 
 366 - atomusdt, 15m, sl: 3, tp: 5, Close, 1000.77, 147.28, 6.79, 25.36% 
 367 - atomusdt, 15m, sl: 3, tp: 9, Close, 1056.57, 147.31, 7.1

In [16]:
results_df.to_csv(f'results_df_007.csv')